In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import StratifiedGroupKFold

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/classification/accoustics/extraction_4.0')
data = pd.read_excel('play_cg_prosody.xlsx',index_col=0)
data = data.dropna()

In [ ]:
data

In [ ]:
X = data.iloc[:,:-1].to_numpy()
y = np.array([0 if x=='no_rad' else 1 for x in data.iloc[:,-1]])
groups = data.index

In [ ]:
groups

In [ ]:
kf = GroupKFold(n_splits=5)
accuracies= []
predictions = []
indexes =[]
ys = []
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y, groups,random_state=42)
for train_index, test_index in kf.split(X_shuffled, y_shuffled, groups_shuffled):

            # scale training and test data based on statistics of only training data
    scaler = StandardScaler()
    X_train_, X_test_ = X[train_index], X[test_index]
    y_train_, y_test_ = y[train_index], y[test_index]
    print("Train :", train_index, "Test :", test_index)
    scaler = StandardScaler()
    scaler.fit(X_train_)
        # normalize data
    X_train_, X_test_ = scaler.transform(X_train_), scaler.transform(X_test_)
    model = LogisticRegression(C=1.623776739188721, class_weight='balanced',
                                     penalty='l2', solver='liblinear')
    model.fit(X_train_, y_train_)
    print(model.predict(X_test_))
    predictions.append(model.predict(X_test_))
    accuracies.append(accuracy_score(y_test_, model.predict(X_test_)))
    indexes.append(groups[test_index])
    ys.append(y_test_)

In [ ]:
mean(accuracies)

In [ ]:
y

In [ ]:
len(np.where(y==1)[0]), len(np.where(y==0)[0])

In [ ]:
import seaborn as sns
best_params_l2 = []
best_scores=[]
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y, groups ,random_state=42)
importance = []
for train, test in kf.split(X_shuffled, y_shuffled, groups_shuffled):

    X_train, X_test, y_train, y_test,groups_shuffled_train, groups_shuffled_test = X_shuffled[train], X_shuffled[test], y_shuffled[train], y_shuffled[test], groups_shuffled[train], groups_shuffled[test]

    pipe = Pipeline([('scaler', preprocessing.StandardScaler()),
                                 ('clf', LogisticRegression(class_weight='balanced'))])
   
    C = [0.0001]
    penalty = ['l2']
    solver = ['liblinear']
    param_grid = [{'clf__solver': solver,
                  'clf__C': C,
                  'clf__penalty': penalty}]
    scoring = 'accuracy'

    gs = GridSearchCV(estimator=pipe, param_grid = param_grid, scoring = scoring,refit=True)
    gs.fit(X_train, y_train)
    importance.append(gs.best_estimator_.named_steps['clf'].coef_[0])
    print(gs.best_estimator_.named_steps['clf'].coef_[0])
    best_score = gs.best_score_
    print ("Fold: {} {:.4f}".format(scoring, best_score))
    print(gs.best_estimator_.named_steps['clf'].predict_proba(X_train))
    best_params_l2.append(gs.best_params_)
    best_scores.append(best_score)

In [ ]:
best_params_l2, best_scores

# run pca on dataset to get the most important features 


In [ ]:
x = StandardScaler().fit_transform(X)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
principalcomponentst = pca.fit_transform(x)

In [ ]:
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))


In [ ]:
principalcomponentst.shape

In [ ]:
df = pd.DataFrame(list(zip([val for sublist in predictions for val in sublist], [val for sublist in indexes for val in sublist],
                          [val for sublist in ys for val in sublist])), columns = ['prediction', 's_id', 'lbl'])

In [ ]:
df.groupby(['s_id']).agg(lambda x:x.value_counts().index[0])

In [ ]:
df.groupby(['s_id']).agg(lambda x:x.value_counts().index[0])

In [ ]:
predictions = []
indexes =[]
ys = []
for train_idx, test_idx in gss.split(X, y, groups):
    X_train_, X_test_ = X[train_idx], X[test_idx]
    y_train_, y_test_ = y[train_idx], y[test_idx]
    print("Train :", train_idx, "Test :", test_idx)
    scaler = StandardScaler()
    scaler.fit(X_train_)
    X_train_, X_test_ = scaler.transform(X_train_), scaler.transform(X_test_)
    model = LogisticRegression(class_weight='balanced')
    model.fit(X_train_, y_train_)
    print(model.predict(X_test_))
    predictions.append(model.predict(X_test_))
    indexes.append(groups[test_idx])
    ys.append(y_test_)

In [ ]:
df = pd.DataFrame(list(zip([val for sublist in predictions for val in sublist], [val for sublist in indexes for val in sublist],
                          [val for sublist in ys for val in sublist])), columns = ['prediction', 's_id', 'lbl'])

In [ ]:
df = pd.DataFrame(list(zip([val for sublist in predictions for val in sublist], [val for sublist in indexes for val in sublist],
                          [val for sublist in ys for val in sublist])), columns = ['prediction', 's_id', 'lbl'])
pd.set_option('display.max_rows', None)
df.groupby(['s_id']).agg(lambda x:x.value_counts().index[0])

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    LogisticRegression(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

In [ ]:
final_res = []
for classifier in classifiers:
    kf = GroupKFold(n_splits=5)
    accuracies= []
    predictions = []
    indexes =[]
    ys = []
    for train_index, test_index in kf.split(X,y, groups):

                # scale training and test data based on statistics of only training data
        scaler = StandardScaler()
        X_train_, X_test_ = X[train_index], X[test_index]
        y_train_, y_test_ = y[train_index], y[test_index]
        print("Train :", train_index, "Test :", test_index)
        scaler = StandardScaler()
        scaler.fit(X_train_)
            # normalize data
        X_train_, X_test_ = scaler.transform(X_train_), scaler.transform(X_test_)
        model = classifier.fit(X_train_, y_train_)
        #print(model.predict(X_test_))
        predictions.append(model.predict(X_test_))
       # accuracies.append(accuracy_score(y_test_, model.predict(X_test_)))
        #print(groups[test_index])
        indexes.append(groups[test_index])
        ys.append(y_test_)

    df = pd.DataFrame(list(zip([val for sublist in predictions for val in sublist], [val for sublist in indexes for val in sublist],
                              [val for sublist in ys for val in sublist])), columns = ['prediction', 's_id', 'lbl'])
    final_res.append(df.groupby(['s_id']).agg(lambda x:x.value_counts().index[0]))

In [ ]:

##!!!!!!!! this is the pipeline!!!LR

best_estimators = []
#accuracies = []

results = []
predictions = []
#X,y, groups = shuffle(X,y,data.index ,random_state=42)
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,groups ,random_state=42)

scalar = preprocessing.StandardScaler()
parameter_candidates = {'model__penalty':['l1'], 'model__solver':['liblinear'],
                        'model__C':np.logspace(-4,4,20)}
clf = LogisticRegression(class_weight='balanced')
pipeline = Pipeline(steps=[('preprocess', scalar), ('model', clf)])

cv = GroupKFold(n_splits=5)
#cross_val_score(pipeline, X, y, cv = cv)
search = GridSearchCV(pipeline, parameter_candidates, scoring = ['accuracy','f1'],refit='f1', cv =cv)

search.fit(X_shuffled, y_shuffled, groups = groups_shuffled)
print(search.best_score_)
print(search.best_estimator_)
#pruint(pd.DataFrame.from_dict(search.cv_results_))
#predictions.append(search.best_estimator_.predict(test_x))

In [ ]:
search.cv_

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,data.index ,random_state=42)
scoring = ['accuracy', 'recall','precision','f1']
scaler = preprocessing.StandardScaler()
clf = LogisticRegression(C=0.0001,class_weight='balanced',penalty='l1', solver='liblinear')
pipe = Pipeline(steps=[("scaler", scaler), ("clf", clf)])
cv = GroupKFold(n_splits=5)


scores = cross_validate(pipe, X_shuffled, y_shuffled,groups=groups_shuffled, cv=cv, scoring=scoring, return_estimator=True)

In [ ]:
scores['test_recall']

In [ ]:
_scores = []
for i in range(0,100):
    y_shuffled,groups_shuffled= shuffle(y,groups,random_state=i)
    scoring = ['accuracy', 'recall','precision','f1']
    scaler = preprocessing.StandardScaler()
    clf = LogisticRegression(C=1.61584821106602,class_weight='balanced',penalty='l2', solver='liblinear')
    pipe = Pipeline(steps=[("scaler", scaler), ("clf", clf)])
    cv = GroupKFold(n_splits=5)
    scores = cross_validate(pipe, X, y_shuffled,groups=groups_shuffled, cv=cv, scoring=scoring)
    _scores.append(np.mean(scores['test_accuracy']))
_scores

In [ ]:
from scipy import stats
def permutation_test(X,y,groups, clf):
    _scores = []
    for i in range(0,100):
        y_shuffled,groups_shuffled= shuffle(y,groups,random_state=i)
        
        scoring = ['accuracy', 'recall','precision','f1']
        scaler = preprocessing.StandardScaler()
        pipe = Pipeline(steps=[("scaler", scaler), ("clf", clf)])
        cv = GroupKFold(n_splits=5)
        scores = cross_validate(pipe, X, y_shuffled,groups=groups_shuffled, cv=cv, scoring=scoring)
        _scores.append(np.mean(scores['test_accuracy']))
    score_no_perm = cross_validate(pipe, X_shuffled, y_shuffled,groups=groups_shuffled, cv=cv, scoring=scoring)
    t_test = stats.ttest_1samp(_scores, np.mean(score_no_perm['test_accuracy']))
    return(_scores,np.mean(score_no_perm['test_accuracy']), t_test.pvalue)


    

In [ ]:
permutation_test(X,y,groups, clf)

In [ ]:
f_importances = [x.named_steps['clf'].coef_[0] for x in  scores['estimator']]
np.mean(f_importances,axis=0)

In [ ]:
features = pd.Series(np.mean(f_importances, axis=0), index=data.columns[:-1])
features

In [ ]:
features.abs().nlargest(20).plot(kind='barh')
plt.savefig('features_child_meal_2.png', bbox_inches='tight')

In [ ]:
### apply t test on features
from itertools import combinations
from scipy.stats import ttest_ind
grps = data['label'].unique()
combs = combinations(grps, 2)

ttests = {
    f'{c1}_{c2}': ttest_ind(
        data.loc[data['label'] == c1, 'f0_mean'], 
        data.loc[data['label'] == c2, 'f0_mean']
    ) for c1, c2 in combs
}

In [ ]:
ttests

In [ ]:
data.groupby('label')['f0_mean'].mean()

In [ ]:
from numpy import std, mean, sqrt
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
cohen_d(rad_abs_jitter['f0_mean'], norad_abs_jitter['f0_mean'])

In [ ]:
rad_abs_jitter = data[['localabsoluteJitter' ,'f0_mean']].loc[data['label'] == 'rad']
norad_abs_jitter = data[['localabsoluteJitter','f0_mean']].loc[data['label'] == 'no_rad']


In [ ]:
scores, np.mean(scores['test_accuracy']), np.mean(scores['test_recall']), np.mean(scores['test_precision'])

In [ ]:
scores

In [ ]:
clf = LogisticRegression(C=1.61584821106602,class_weight='balanced',penalty='l2', solver='liblinear')
pipe = Pipeline([('scaler', StandardScaler()), ('clf',clf)])
cv = GroupKFold(n_splits=5)
score, permutation_score, pval = permutation_test_score(pipe,X_shuffled, y_shuffled,groups=groups_shuffled,
                                                        n_permutations = 100, cv = cv,scoring="f1")

In [ ]:
score, permutation_score, pval

In [ ]:

##!!!!!!!! this is the pipeline!!!

best_estimators = [] 
#accuracies = []

results = []
predictions = []
#X,y, groups = shuffle(X,y,data.index ,random_state=42)

X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,groups ,random_state=42)

scalar = preprocessing.StandardScaler()
parameter_candidates = {'model__C':[0.001, 0.01, 0.1, 1, 10, 100], 'model__gamma':[0.001, 0.01, 0.1, 1, 10, 100],
                        'model__kernel':['rbf']}
clf = SVC(class_weight='balanced')
pipeline = Pipeline(steps=[('preprocess', scalar), ('model', clf)])

cv = GroupKFold(n_splits=5)
#cross_val_score(pipeline, X, y, cv = cv)
search = GridSearchCV(pipeline, parameter_candidates, scoring = 'accuracy', cv =cv)

search.fit(X_shuffled, y_shuffled, groups = groups_shuffled)
print(search.best_score_)
print(search.best_estimator_)
#pruint(pd.DataFrame.from_dict(search.cv_results_))
#predictions.append(search.best_estimator_.predict(test_x))

In [ ]:
## idea: shift the labels between groups not within groups 

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,data.index ,random_state=42)
scoring = ['accuracy', 'recall','precision', 'f1']
scaler = preprocessing.StandardScaler()
clf = SVC(kernel='rbf',C=0.001, gamma= 0.001, class_weight='balanced')
pipe = Pipeline(steps=[("scaler", scaler), ("classifier", clf)])
cv = StratifiedGroupKFold(n_splits=5)

scores = cross_validate(pipe, X_shuffled, y_shuffled,groups=groups_shuffled, cv=cv, scoring=scoring)

In [ ]:
scores

In [ ]:
cv =GroupKFold(n_splits=5)
i=1
for train, test in cv.split(y,groups=groups_shuffled):
    print('Fold:',i,"Train:",np.unique(y[train]),'group train:',np.unique(groups_shuffled[train]),'group test:',np.unique(groups_shuffled[test]), "Test:",np.unique(y[test]))
    i+=1

In [ ]:
np.mean(scores['test_accuracy']), np.mean(scores['test_recall']), np.mean(scores['test_precision']), np.mean(scores['test_f1'])

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
cv = GroupShuffleSplit(n_splits=10, train_size=.8, random_state=42)
clf = LogisticRegression(C=1.61584821106602,class_weight='balanced',penalty='l2', solver='liblinear')
pipe = Pipeline([('scaler', StandardScaler()), ('clf', clf)])
score, permutation_score, pval = permutation_test_score(pipe,X_shuffled, y_shuffled,groups=groups_shuffled,
                                                        n_permutations = 100, cv = cv,scoring="accuracy")


In [ ]:
score, permutation_score, pval

In [ ]:
### group by group and shuffle labels:
original_grouping_vector = data.index
unique_values, indexes, inverse = np.unique(original_grouping_vector, return_inverse=True, return_index=True)
new_grouping_vector = indexes[inverse] # This is where the magic happens!

splitter = GroupKFold()
for train, test in splitter.split(X, y, groups=new_grouping_vector):
    print(data.iloc[test, :])

In [ ]:
### group by group and shuffle labels:
original_grouping_vector = data.index
unique_values, indexes, inverse = np.unique(original_grouping_vector, return_inverse=True, return_index=True)
new_grouping_vector = indexes[inverse] # This is where the magic happens!
## inverse are the indices of the duplicated indices in original array 
## indexes are the indices of the unique values in original array
splitter = GroupKFold()
for train, test in splitter.split(X, y, groups=new_grouping_vector):
    print(data.iloc[test, :],y[test])

In [ ]:
### shuffle the y's but keep them consistent within groups 
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=42)
for train_index, test_index in gss.split(X, y, groups):
    print("TRAIN:", np.unique(groups[train_index]), "TEST:", np.unique(groups[test_index]))

In [ ]:
len(np.unique(groups))

In [ ]:
np.unique(groups)

In [ ]:
pval, score

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('clf',logistic)])
cv = KFold(n_splits=5)
score, permutation_score, pval = permutation_test_score(pipe,X_shuffled, y_shuffled,
                                                        n_permutations = 100, cv = cv,scoring="accuracy")

In [ ]:
score, permutation_score

In [ ]:
scores

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,data.index ,random_state=42)
scoring = ['accuracy', 'recall','precision']
scaler = preprocessing.StandardScaler()
clf = SVC(kernel='rbf',C=0.001,gamma= 0.001, class_weight='balanced')
pipe = Pipeline(steps=[("scaler", scaler), ("classifier", clf)])
cv = GroupKFold(n_splits=5)

scores = cross_validate(pipe, X_shuffled, y_shuffled,groups_shuffled, cv=cv, scoring=scoring)

In [ ]:
data